# Draft analysis 

---

Group name: Gruppe D

---


## 1. Introduction

### 1.1 Motivation

In dieser Untersuchung setzt sich die Projektgruppe D mit dem Thema Mobilität außeinander. Dieses Thema ist im Hinblick auf aktuelle gesellschaftliche und politische Problemstellungen bezüglich Klimaerwärmung und Klimazielen sehr relevant. Zürich ist neben Wien eine der am besten ausgebauten Städte bezogen auf den öffentlichen Personenverkehr. Es liegt eine Umfangreiche Datensammlung der Verkehrsbezüge Zürich vor, mit der das bestehende Verkersangebot untersucht wird.

### 1.2 Data
Zur Analyse der Daten stehen wöchentliche csv-Dateien zur Verfügung, welche eine Vielzahl an Daten beinhalten. Jeder Eintrag stellt einen geplanten Halt eines Fahrzeugs der Verkehrsbetriebe Zürich dar und gibt eine präzise Auskunft über Datum, Linie, Fahrzeug, Haltestelle und vielem mehr. Somit ermöglicht der Datensatz einen Vergleich zwischen den geplanten und tatsächlich erfolgten Fahrzeiten der Fahrzeuge der Verkehrsbetriebe Zürich. Da vielen Daten aus der Tabelle eine interne ID und somit einen Fremdschlüssel zu anderen Tabellen darstellen, sind weitere Tabellen vorhanden, um diese Daten mit konkreten, verständlichen Bezeichnungen verknüpfen zu können.

Der Datenlieferant ist Verkehrsbetriebe Zürich, Departement der Industriellen Betriebe. Die Daten wurden erstmals 2015 erfasst und werden seither wöchentlich sonntags aktualisiert. Sämtliche Zeiten werden dabei auf die Sekunde gemessen. Über die genaue Bereitstellung der Daten gibt es keine Informationen. Da die Daten wöchentlich aktualisiert werden, ist davon auszugehen, dass die Daten durch einen automatischen Export aus einer oder mehreren weiteren Datenbanktabellen zusammengetragen und miteinander verknüpft werden.

Für die Untersuchung werden Daten vom September 2022 betrachtet.

### 1.3 Research Question
Aus der herangegangenen Motiation und Darlegung der Daten stellt sich die Frage: Wie kann der bestehnde Nahverkehr in Zürich verbessert werden? 

#### 1.3.1 Hypothese

### 1.4 Data Dictionary

## 2. Setup

In [3]:
import pandas as pd
import altair as alt

## 3. Data

### 3.1 Import data

Import der benötigten CSV-Dateien

In [65]:
# import csv's

# 1. Septemberwoche 2022
df_week1 = pd.read_csv("https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/download/Fahrzeiten_SOLL_IST_20220904_20220910.csv")

# 2. Septemberwoche 2022
df_week2 = pd.read_csv("https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/download/Fahrzeiten_SOLL_IST_20220911_20220917.csv")

# 3. Septemberwoche 2022
df_week3 = pd.read_csv("https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/download/Fahrzeiten_SOLL_IST_20220918_20220924.csv")

# 4. Septemberwoche 20222
df_week4 = pd.read_csv("https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/download/Fahrzeiten_SOLL_IST_20220925_20221001.csv")

# Haltestellen
df_haltestellen = pd.read_csv('https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/download/Haltestelle.csv')

# Haltepunkte
df_haltepunkte = pd.read_csv('https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/download/Haltepunkt.csv')

# Linien
df_linie = pd.read_csv('https://data.stadt-zuerich.ch/dataset/vbz_fahrgastzahlen_ogd/download/LINIE.csv', delimiter=';')


kleine daten Übersicht des Haupt-DataFrames 

In [66]:
df_week1.head()

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,...,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach
0,2,1,08.09.22,3059,1,25,440,0,BELL,08.09.22,...,173050,1,1,1,SGEI - BTIE,284307,1565,2104,52890,10538
1,2,1,08.09.22,2069,4,25,440,0,BELL,08.09.22,...,173050,1,1,1,SGEI - BTIE,284477,1565,2104,52890,10538
2,2,1,08.09.22,3059,1,25,440,0,BELL,08.09.22,...,173050,1,1,1,SGEI - BTIE,284307,1565,2104,52890,10538
3,2,1,08.09.22,2079,10,25,440,0,BELL,08.09.22,...,173050,1,1,1,SGEI - BTIE,284540,1565,2104,52890,10538
4,2,1,08.09.22,2066,2,25,440,0,BELL,08.09.22,...,173050,1,1,1,SGEI - BTIE,284361,1565,2104,52890,10538


### 3.2 Data structure

In [67]:
df_week1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1423178 entries, 0 to 1423177
Data columns (total 34 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   linie                 1423178 non-null  int64 
 1   richtung              1423178 non-null  int64 
 2   betriebsdatum         1423178 non-null  object
 3   fahrzeug              1423178 non-null  int64 
 4   kurs                  1423178 non-null  int64 
 5   seq_von               1423178 non-null  int64 
 6   halt_diva_von         1423178 non-null  int64 
 7   halt_punkt_diva_von   1423178 non-null  int64 
 8   halt_kurz_von1        1423178 non-null  object
 9   datum_von             1423178 non-null  object
 10  soll_an_von           1423178 non-null  int64 
 11  ist_an_von            1423178 non-null  int64 
 12  soll_ab_von           1423178 non-null  int64 
 13  ist_ab_von            1423178 non-null  int64 
 14  seq_nach              1423178 non-null  int64 
 15

### 3.3 Data corrections

In [71]:
df_week1 = df_week1.drop(['kurs', 'seq_von', 'datum_von', 'halt_diva_von','halt_kurz_von1', 'halt_punkt_diva_von','seq_nach', 'halt_diva_nach', 'halt_punkt_diva_nach', 'halt_kurz_nach1', 'datum_nach', 'fahrt_id', 'fahrweg_id', 'fw_no', 'fw_typ', 'fw_kurz', 'fw_lang', 'umlauf_von'], axis=1)

df_week1.head()

,linie,richtung,betriebsdatum,fahrzeug,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach
0,2,1,08.09.22,3059,30618,30603,30636,30632,30702,30713,30732,30740,1565,2104,52890,10538
1,2,1,08.09.22,2069,26118,26053,26136,26069,26202,26139,26232,26158,1565,2104,52890,10538
2,2,1,08.09.22,3059,36462,36409,36486,36442,36564,36528,36588,36553,1565,2104,52890,10538
3,2,1,08.09.22,2079,34662,34656,34686,34675,34764,34770,34788,34787,1565,2104,52890,10538
4,2,1,08.09.22,2066,31068,31100,31086,31121,31152,31220,31182,31266,1565,2104,52890,10538


In [77]:
df_week1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1423178 entries, 0 to 1423177
Data columns (total 16 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   linie               1423178 non-null  int64 
 1   richtung            1423178 non-null  int64 
 2   betriebsdatum       1423178 non-null  object
 3   fahrzeug            1423178 non-null  int64 
 4   soll_an_von         1423178 non-null  int64 
 5   ist_an_von          1423178 non-null  int64 
 6   soll_ab_von         1423178 non-null  int64 
 7   ist_ab_von          1423178 non-null  int64 
 8   soll_an_nach        1423178 non-null  int64 
 9   ist_an_nach1        1423178 non-null  int64 
 10  soll_ab_nach        1423178 non-null  int64 
 11  ist_ab_nach         1423178 non-null  int64 
 12  halt_id_von         1423178 non-null  int64 
 13  halt_id_nach        1423178 non-null  int64 
 14  halt_punkt_id_von   1423178 non-null  int64 
 15  halt_punkt_id_nach  1423178 non-

In [78]:
df_haltestellen_new = df_haltestellen.drop(['halt_diva', 'halt_kurz', 'halt_ist_aktiv'], axis=1)
df_haltestellen_new.head()

,halt_id,halt_lang
0,143,"Birmensdorf ZH, Sternen/WSL"
1,309,"Waldegg, Birmensdorferstrasse"
2,373,"Zürich Flughafen, Fracht"
3,539,"Thalwil, Bahnhof"
4,588,"Zürich Flughafen, Bahnhof"


In [79]:
# join both dataframes
df_all = df_week1.join(df_haltestellen_new.set_index('halt_id'), on='halt_id_von')
df_all = df_all.rename(columns={"halt_lang": "halt_lang_von"})

df_all = df_all.join(df_haltestellen_new.set_index('halt_id'), on='halt_id_nach')
df_all = df_all.rename(columns={"halt_lang": "halt_lang_nach"})

df_all.head()

,linie,richtung,betriebsdatum,fahrzeug,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_lang_von,halt_lang_nach
0,2,1,08.09.22,3059,30618,30603,30636,30632,30702,30713,30732,30740,1565,2104,52890,10538,"Zürich, Bellevue","Zürich, Opernhaus"
1,2,1,08.09.22,2069,26118,26053,26136,26069,26202,26139,26232,26158,1565,2104,52890,10538,"Zürich, Bellevue","Zürich, Opernhaus"
2,2,1,08.09.22,3059,36462,36409,36486,36442,36564,36528,36588,36553,1565,2104,52890,10538,"Zürich, Bellevue","Zürich, Opernhaus"
3,2,1,08.09.22,2079,34662,34656,34686,34675,34764,34770,34788,34787,1565,2104,52890,10538,"Zürich, Bellevue","Zürich, Opernhaus"
4,2,1,08.09.22,2066,31068,31100,31086,31121,31152,31220,31182,31266,1565,2104,52890,10538,"Zürich, Bellevue","Zürich, Opernhaus"


In [80]:
# Calculate the punctuality per line
df_all['punct_cat'] = df_all.apply(lambda x: 'delay' if x["ist_an_nach1"] - x["soll_an_nach"] >= 120 else 'too early'
                                            if x["ist_ab_nach"] - x["soll_ab_nach"]<= -60 else "punctual", axis=1)

df_all.head()

,linie,richtung,betriebsdatum,fahrzeug,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_lang_von,halt_lang_nach,punct_cat
0,2,1,08.09.22,3059,30618,30603,30636,30632,30702,30713,30732,30740,1565,2104,52890,10538,"Zürich, Bellevue","Zürich, Opernhaus",punctual
1,2,1,08.09.22,2069,26118,26053,26136,26069,26202,26139,26232,26158,1565,2104,52890,10538,"Zürich, Bellevue","Zürich, Opernhaus",too early
2,2,1,08.09.22,3059,36462,36409,36486,36442,36564,36528,36588,36553,1565,2104,52890,10538,"Zürich, Bellevue","Zürich, Opernhaus",punctual
3,2,1,08.09.22,2079,34662,34656,34686,34675,34764,34770,34788,34787,1565,2104,52890,10538,"Zürich, Bellevue","Zürich, Opernhaus",punctual
4,2,1,08.09.22,2066,31068,31100,31086,31121,31152,31220,31182,31266,1565,2104,52890,10538,"Zürich, Bellevue","Zürich, Opernhaus",punctual


## 4. Analysis

### 4.1 Descriptive statistics

### 4.2 Exploratory data analysis

## 5. Visualizations

### 5.1 Visualization ideas

### 5.2 Save Visualizations



Save your draft visualizations in the folder `reports/visualizations/`. Use a meaningful name (always include the word `draft` and a `timestamp`in your filename).

## 6. Conclusion and recommended action